In [1]:
#!/usr/bin/env python
# coding: utf-8
"""
Author: Mrinal Kanti Dhar
December 12, 2024
"""

# * v6: Color blending added
# * v7: Parameter keep_only is changed to keep_range
# * v7_2: Predictions are read from the excel file
# * v7_3: Generate confidence score based on PCA/t-SNE and model outputs

print('************ The code is loaded ************')

import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

# Append paths
import sys
import os
sys.path.append(os.getcwd() + '/networks/') 
sys.path.append(os.getcwd() + '/utils/') 
sys.path.append(os.getcwd() + '/dataloader/') 
sys.path.append(os.getcwd() + '/losses/') 
sys.path.append(os.getcwd() + '/config/') 

from read_ultrasound import read_nib

import cv2
from dataloader.loader import MyDataset
import torch
from torch import nn
# from torch.utils.tensorboard import SummaryWriter
import numpy as np
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from datetime import datetime
from copy import deepcopy
import matplotlib.pyplot as plt
import pandas as pd
import time
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

# import albumentations as A

from networks import nets
from network_parameters.params import model_params
from utils import augmentations, normalization
from losses.loss import loss_func

# from sklearn.metrics import confusion_matrix, classification_report
# from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score
# from sklearn.metrics import roc_curve, RocCurveDisplay, auc
# from sklearn.preprocessing import label_binarize
# from sklearn.metrics import roc_auc_score
# from sklearn.model_selection import StratifiedKFold

from box import Box
import yaml
from tqdm import tqdm
import argparse

************ The code is loaded ************


In [2]:
# # ### Function to read config file from command line
# def get_config_from_args():
#     parser = argparse.ArgumentParser(description="Pass config file")
#     parser.add_argument('--config', type=str, required=True, help="Path to the YAML config file")
#     args = parser.parse_args()
#     return args
    
# # ### Read config file
# # Get the config file from command-line arguments
# args = get_config_from_args()

# with open(args.config, "r") as file:
#     config = yaml.safe_load(file)
# config = Box(config)

### Parameters

In [ ]:
#%% Parameters
BASE_MODEL = "EnsembleResNet18Ft512_EfficientNetB2SFt1408" # config.model.name

base_model_name = "EnsembleResNet18Ft512_EfficientNetB2SFt1408_2024-10-25_17-31-23" # config.test.base_model_name

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

LR = 0.001 # config.train.lr #0.0001 # learning rate
WEIGHT_DECAY = 0.001 # config.train.weight_decay #1e-5

BATCH_SIZE = 1 # config.train.batch_size
ONE_HOT = True # config.train.one_hot
N_CLASSES = 2 # config.train.n_classes
ONLY_ADNEXAL = False # config.data.only_adnexal
ONLY_FLUID = True # config.data.only_fluid
ONLY_SOLID = True # config.data.only_solid
DRAW_BBOX = False # config.data.draw_bbox
CROP_ROI = True # config.data.crop_roi
MARGIN = 200 # config.data.margin
RESIZE = True # config.data.resize
KEEP_ASPECT_RATIO = True # config.data.keep_aspect_ratio
TARGET_SIZE = [224,224] # config.data.target_size
CONCAT = ["image", "fluid", "solid"] # config.data.concat # Possible keywords are: "image", "adnexal", "fluid", "solid", "mask"
INPUT_CH = len(CONCAT)

N_FOLDS = 5

# Parameters for ensemble models
DROPOUT = 0.3 # config.model.dropout
OUT_CHS = [5376, 1024, 512, 256] # config.model.out_channels # concat feature maps will be converted to OUT_CHS

### Image directory

In [4]:
root = "/research/m324371/Project/adnexal/" # config.directories.root
result_dir = "/research/m324371/Project/adnexal/results/" # config.directories.result_dir
train_im_dir = "/research/m324371/Project/adnexal/dataset/train/" # config.directories.train_im_dir
val_im_dir = "/research/m324371/Project/adnexal/dataset/train/" # config.directories.val_im_dir
test_im_dir = "/research/m324371/Project/adnexal/dataset/test/" # config.directories.test_im_dir

### Read the test result excel file

In [5]:
test_results = pd.read_excel(os.path.join(result_dir, base_model_name, "results_test", base_model_name + "_avg.xlsx"))

### Transforms

In [6]:
# Normalization
normalize_transform = normalization.normalize_v2(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # always normalize

# Augmentation
transform = augmentations.transforms()

### Prepare preprocessing dictionary

In [7]:
pp_dict = {}
pp_dict["only_adnexal"] = ONLY_ADNEXAL
pp_dict["only_fluid"] = ONLY_FLUID
pp_dict["only_solid"] = ONLY_SOLID
pp_dict["draw_bbox"] = DRAW_BBOX
pp_dict["crop_roi"] = CROP_ROI
pp_dict["margin"] = MARGIN
pp_dict["resize"] = RESIZE
pp_dict["keep_aspect_ratio"] = KEEP_ASPECT_RATIO
pp_dict["target_size"] = TARGET_SIZE

# For training data
train_pp_dict = pp_dict.copy()
train_pp_dict["file_dir"] = train_im_dir

# For validation data
val_pp_dict = pp_dict.copy()
val_pp_dict["file_dir"] = val_im_dir

# For test data
test_pp_dict = pp_dict.copy()
test_pp_dict["file_dir"] = test_im_dir

In [ ]:
# ### Read adnexal_dataset.xlsx
# Read adnexal dataset.xlsx
df_location = '/research/m324371/Project/adnexal/adnexal_dataset_all.xlsx' # config.directories.excel_dir
dataframe = pd.read_excel(df_location, sheet_name=None) 

# Read train and test sheets
df_train = dataframe['train']  
df_test = dataframe['test']  

print(df_train.head())

train_names = df_train['Base names']
train_class = df_train['Class']

### Pick test images

In [9]:
# # Uncomment if you want selected test images by their indices
# test_idx = [66, 114, 160]
# df_test = df_test.iloc[test_idx]
# df_test.reset_index(drop=True, inplace=True) # Reset index. Start from 0

In [ ]:
test_names = list(df_test["Base names"])

print('Test names:', test_names)

### Dataloader

In [12]:
train_dataset = MyDataset(
            df_train, 
            n_classes=N_CLASSES, 
            transform=None, 
            normalize=normalize_transform,
            one_hot=ONE_HOT,
            preprocess_dict=train_pp_dict,
            concat=CONCAT,
            )
    
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=1)

test_dataset = MyDataset(
            df_test, 
            n_classes=N_CLASSES, 
            transform=None, 
            normalize=normalize_transform,
            one_hot=ONE_HOT,
            preprocess_dict=test_pp_dict,
            concat=CONCAT,
            )
    
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=1)

### Base model

In [13]:
# Dynamically get the model class from nets
# get_model = getattr(nets, config.model.name) # get_model is a model class, not an object

# params = model_params(config.model.name, config) # initialize the model with other parameters 

# base_model = get_model(**params)

get_model = getattr(nets, BASE_MODEL) # get_model is a model class, not an object
params = dict()
params["num_classes"] = N_CLASSES
params["out_channels"] = OUT_CHS
params["pretrain"] = True
params["dropout"] = DROPOUT 
params["in_chs"] = len(CONCAT) # len(config.data.concat) # <<<<<<<<<<<<<<<<<<<<<<<<<<<<< temporarily changed for doppler
params["separate_inputs"] = 3 # config.model.separate_inputs
base_model = get_model(**params)

### Test save directory

In [14]:
# base_model_name = 'resnet18_imsize256_2024-09-25_16-55-17'
test_save_dir = os.path.join(result_dir, base_model_name, 'results_test', 'similarity_test_v7-3_top5')
os.makedirs(test_save_dir, exist_ok=True)

### Load model

In [ ]:
# Find the best model name from the k-fold summary report
dir_excel = os.path.join(result_dir, base_model_name, 'results_val')
file_name = base_model_name + '_val.xlsx'

dataframe = pd.read_excel(os.path.join(dir_excel, file_name), sheet_name=None)

metric, folds = [], []

for sheet_name, dframe in dataframe.items():
    first_row = dframe.iloc[0]
    metric.append(first_row["Accuracy"])
    folds.append(sheet_name)
                  
best_metric_idx = np.argmax(metric)
best_model_name = dataframe[folds[best_metric_idx]]["Model name"][0]
best_epoch = dataframe[folds[best_metric_idx]]["Best epoch"][0]

print('Base model name:', base_model_name)
print('Best model index:', best_metric_idx)
print('Best epoch:', best_epoch)
print('Best model name:', best_model_name)

In [ ]:
# Get all model names
model_names = []
for fold in folds:
    model_name = dataframe[fold]["Model name"][0]
    model_names.append(model_name)

print(model_names)

# Define hook function globally or outside the loop
features = []  # Global list to store features

def hook(module, input, output):
    features.append(output.clone().detach().cpu())  # Store hooked output safely

# Attach hooks to models and process
trained_models = []

for model_name in model_names:
    checkpoint_loc = os.path.join(result_dir, base_model_name, 'checkpoints', model_name)
    checkpoint = torch.load(os.path.join(checkpoint_loc, 'best_model.pth'))

    # Make a deep copy of the base model
    model_copy = deepcopy(base_model)

    # Load the weights into the copied model
    model_copy.load_state_dict(checkpoint['state_dict'])
    model_copy.eval()  # Set the copied model to evaluation mode

    # Attach hook to capture features from an intermediate layer
    layer_to_hook = model_copy.classification.avg_pool # layer to hook <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    layer_to_hook.register_forward_hook(hook)

    # Append the copied model to the list of trained models
    trained_models.append(model_copy.to(DEVICE))

print(f"No. of models: {len(trained_models)}")

### Generate features

#### For training data

In [ ]:
# For train data
store_train_features = []  # Store features for all training images
with torch.no_grad():
    for i, (inputs, labels, names) in tqdm(enumerate(train_loader)):
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        if ONE_HOT:
            labels = torch.argmax(labels, dim=1).data.cpu().numpy().tolist()
        else:
            labels = labels.data.cpu().numpy().squeeze()

        store_pred = []
        for model_idx, model in enumerate(trained_models):

            features.clear()

            # Forward pass
            outputs_ = model(inputs)

            # Handle hooked features
            if len(features) > 0:
                hooked_features = features[0].numpy()
                for b in range(hooked_features.shape[0]):  # Iterate over batch
                    hooked_features_batch = hooked_features[b].squeeze().tolist()
                    store_pred.append([names[b], labels[b], hooked_features_batch, model_idx]) # stores per-model
            else:
                print("Warning: Hook did not capture training features")

        store_train_features.append(store_pred) # stores for all models

#### For testing data

In [ ]:
# For test data
store_test_features = []  # Store features for all test images
with torch.no_grad():
    for i, (tinputs, tlabels, tnames) in tqdm(enumerate(test_loader)):
        tinputs = tinputs.to(DEVICE)
        tlabels = tlabels.to(DEVICE)

        if ONE_HOT:
            tlabels = torch.argmax(tlabels, dim=1).data.cpu().numpy().tolist()
        else:
            tlabels = tlabels.data.cpu().numpy().squeeze().tolist()

        store_pred = []
        for model_idx, model in enumerate(trained_models):
            features.clear()

            # Forward pass
            toutputs = model(tinputs)

            # Probabilities 
            prob = torch.softmax(toutputs, dim=1) if ONE_HOT else torch.sigmoid(toutputs)

            # Get predictions
            if ONE_HOT:
                pred_class = torch.argmax(torch.softmax(prob, dim=1), dim=1).data.cpu().numpy().tolist()
            else:
                # toutputs.size(1) == 1:  # Binary classification
                pred_class = (torch.sigmoid(prob) > 0.5).data.cpu().numpy().squeeze().tolist()

            # Handle hooked features
            if len(features) > 0:
                hooked_features = features[0].numpy()
                for b in range(hooked_features.shape[0]):  # Iterate over batch
                    hooked_features_batch = hooked_features[b].squeeze().tolist()
                    store_pred.append([tnames[b], tlabels[b], hooked_features_batch, pred_class[b], model_idx])  # stores per-model
            else:
                print("Warning: Hook did not capture test features")

        store_test_features.append(store_pred)  # stores all models


### Similarity scores

In [ ]:
from collections import Counter
# Lists to store results
detailed_results = []
average_scores = []

# Iterate over each test sample
for test_features in tqdm(store_test_features):  # Features for one test sample
    test_name = test_features[0][0]  # Extract the test image name
    test_label = test_features[0][1] # Extract true label of the test image
    # test_pred = test_features[0][3] # Extract test prediction %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    # predictions_ = [test_features[i][3] for i in range(N_FOLDS)] # len(train_features) is basically no. of models. Because it is a list that contains lists equal to no. models.
    # frequency = Counter(predictions_)
    # test_pred = frequency.most_common(1)[0][0]
    test_pred = int(test_results.loc[test_results["Names"]==test_name]["Prediction"].values)

    # Initialize accumulators for averaging scores
    total_cosine_sim = 0
    total_mse = 0
    num_comparisons = 0

    for train_features in store_train_features:  # Features for one training sample
        train_name = train_features[0][0]  # Extract the train image name
        train_label = train_features[0][1] # Extract train image label

        # Calculate cosine similarity and MSE for outputs from all models
        for i in range(N_FOLDS):  # Assuming 5 models. 
            # len(train_features) is basically no. of models. Because it is a list that contains lists equal to no. models.
            train_output = np.array(train_features[i][2])
            test_output = np.array(test_features[i][2])

            # Cosine Similarity
            cos_sim = np.dot(train_output, test_output) / (
                np.linalg.norm(train_output) * np.linalg.norm(test_output)
            )

            # Mean Squared Error (MSE)
            mse = np.mean((train_output - test_output) ** 2)

            # Accumulate scores for averaging
            total_cosine_sim += cos_sim
            total_mse += mse
            num_comparisons += 1

            # Store individual comparison results
            detailed_results.append({
                "Test_Image": test_name,
                "Train_Image": train_name,
                "Test_Label": test_label,
                "Test_Prediction": test_pred,
                "Train_Label": train_label,
                "Model_Index": i,
                "Cosine_Similarity": cos_sim,
                "MSE": mse
            })

    # Store average scores for the current test image
    average_scores.append({
        "Test_Image": test_name, # >>>>>>>>>>> for batchsize=1, test_name is like ('PN0253_34_20190312',). So, put test_name[0]
        "Average_Cosine_Similarity": total_cosine_sim / num_comparisons,
        "Average_MSE": total_mse / num_comparisons
    })

# Convert results to DataFrames
detailed_results_df = pd.DataFrame(detailed_results)
average_scores_df = pd.DataFrame(average_scores)

# Save results to an Excel file with two sheets
output_filename = os.path.join(test_save_dir, "train_test_similarity_" + datetime.now().strftime('%Y-%m-%d_%H-%M-%S') + ".xlsx")
with pd.ExcelWriter(output_filename, engine='xlsxwriter') as writer:
    detailed_results_df.to_excel(writer, sheet_name='Detailed Results', index=False)
    average_scores_df.to_excel(writer, sheet_name='Average Scores', index=False)

print(f"Results saved to {output_filename}")

### Filter data by the patient name and model index

In [20]:
# Read image
def read_image(file_dir, img_name):
    data = read_nib(os.path.join(file_dir, img_name), target_orientation = np.array([[1, -1], [0, -1], [2, -1]]))
    data = data.squeeze().astype('uint8')
    return data

In [21]:
def color_blend(image, mask, mask_color:list=[255, 0, 0], transparency_factor:float=0.5):
    """
    Args
    ================
    image: Shape HxWxCh or HxW. 
    mask: Shape HxW.
    mask_color: Color that will be assigned to the mask.
    transparency_factor: Controls transparency. 
    """

    # Convert the grayscale image to a 3-channel RGB image for overlay
    if len(image.shape)==2 or image.shape[-1]==1: image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

    # Create a color for the mask overlay
    mask_color = np.array(mask_color)  # Red color in RGB

    # Create a color overlay using the binary mask
    color_overlay = np.zeros_like(image, dtype=np.uint8)
    color_overlay[mask > 0] = mask_color
    
    # Blend the image with the color overlay
    blended = cv2.addWeighted(image, 1 - transparency_factor, color_overlay, transparency_factor, 0)

    return blended


In [22]:
def post_similarity(similarity_df, patient_name, n_folds,
                    train_im_dir, test_im_dir, save_dir,
                    sort_by:str='Cosine_Similarity', keep_range:tuple=(0,4),  
                    blend:bool=True, visualize:bool=False):

    "Part I: Store top similarities for each model in a dataframe."
    # No. of training images to keep
    n_imgs = keep_range[1] - keep_range[0] + 1
    
    # Create model indices. For 5-fold CV, [0,1,2,3,4]
    model_inds = np.arange(0, n_folds).tolist()
    
    # Create a dataframe to store top similarities for each model
    columns = similarity_df.columns.tolist() 
    top_similarity_df = pd.DataFrame(columns=columns)
    
    # Load test image
    test_image = read_image(test_im_dir, patient_name + "_image.nii.gz")
       
    for model_idx in model_inds:
        # Filter dataframe based on patient name and model index
        filtered_rows = similarity_df[(similarity_df['Test_Image'] == patient_name) & (similarity_df['Model_Index'] == model_idx)]
    
        # Check if the range is valid
        length = len(filtered_rows)
        if keep_range[0] < 0 or keep_range[1] >= length:
            raise ValueError(f"Invalid range. keep_range should be between 0 to {length-1} inclusive.")
    
        # Sort data in ascending or descending order
        if sort_by == 'Cosine_Similarity': ascending = False
        elif sort_by == 'MSE': ascending = True
        else: raise ValueError("Unsupported name for the argument sort_by. Supported names are Cosine_Similarity and MSE.")
            
        sorted_filtered_rows = filtered_rows.sort_values(by=sort_by, ascending=ascending).reset_index(drop=True)
        
        # Keep top rows only
        keep_only_sorted_filtered_rows = sorted_filtered_rows.iloc[keep_range[0]:keep_range[1]+1] # 1 added because end is exclusive
    
        top_similarity_df = pd.concat([top_similarity_df, keep_only_sorted_filtered_rows], ignore_index=True)
    
    
    "Part II: Plot the test image and top similarities in the training set at the image level."
    # Create a grid for subplots: 1 row per model
    fig, axes = plt.subplots(nrows=n_folds + 1, ncols=n_imgs+1, figsize=(20, 6 * n_folds))
    fig.suptitle(f"Similarity Results for {patient_name}", fontsize=20, y=0.9)
    
    # Ensure axes is always 2D for consistency
    if n_folds == 1:
        axes = np.expand_dims(axes, axis=0)
    
    # Group data based on model index
    grouped = top_similarity_df.groupby("Model_Index")
    
    # Test class
    test_label = top_similarity_df["Test_Label"].iloc[0]
    test_pred = top_similarity_df["Test_Prediction"].iloc[0]
    
    # Load test mask
    if blend: 
        mask_color = [255, 0, 0]
        transparency_factor = 0.1
        test_mask = read_image(test_im_dir, patient_name + "_mask.nii.gz")
        test_image = color_blend(test_image, test_mask, mask_color=mask_color, transparency_factor=transparency_factor)
    
    # Iterate over groups
    for row_pointer, (key, group) in enumerate(grouped):  # Unpack group into key and DataFrame. key is the Model_Index
        # Plot the test image
        ax = axes[row_pointer, 0]
        ax.imshow(test_image, cmap='gray') if not blend else ax.imshow(test_image)
        ax.set_title(f"Test Image, GT: {test_label}, Pred: {test_pred}, CV: {key + 1}")  
        ax.axis('off')
    
        # Iterate through the rows of the group DataFrame
        for column_pointer, (train_img_name, train_label, model_idx) in enumerate(zip(group["Train_Image"], group["Train_Label"], group["Model_Index"])):
            train_image = read_image(train_im_dir, train_img_name + "_image.nii.gz")
            if blend: 
                train_mask = read_image(train_im_dir, train_img_name + "_mask.nii.gz")
                train_image = color_blend(train_image, train_mask, mask_color=mask_color, transparency_factor=transparency_factor)
    
            ax = axes[row_pointer, column_pointer + 1]  # First column is for the test image
            ax.imshow(train_image, cmap='gray') if not blend else ax.imshow(train_image)
            # ax.set_title(f"Train Image {column_pointer + 1}, Label: {train_label}, CV: {model_idx + 1}")
            ax.set_title(f"{train_img_name}\n Label: {train_label}, CV: {model_idx + 1}")
            ax.axis('off')
    
    "Part III: Find top patient-level similarities from the image-level similarities found in Part II"
    # Sort top_similarity_df
    sorted_top_similarity_df = top_similarity_df.sort_values(by=sort_by, ascending=ascending).reset_index(drop=True)
        
    # Extract patients name from Train_Image and create a new column for it.
    # Image name format: patientName_xx_xxxxx
    sorted_top_similarity_df["Train_Patient_Name"] = sorted_top_similarity_df["Train_Image"].str.split('_').str[0]
    sorted_top_similarity_df.reset_index(drop=True, inplace=True)
    
    # Drop duplicate patients and keep first entry only. 
    filtered_sorted_top_similarity_df = sorted_top_similarity_df.drop_duplicates(subset='Train_Patient_Name', keep='first').reset_index(drop=True)

    
    keep_only_filtered_sorted_top_similarity_df = filtered_sorted_top_similarity_df.head(n_imgs)
    
    #%% Plot the patient-level top similarities
    # Plot the test image
    ax = axes[row_pointer+1, 0]
    ax.imshow(test_image, cmap='gray')
    ax.set_title(f"Test Image, GT: {test_label}, Pred: {test_pred}")  
    ax.axis('off')
    
    # Plot patient-level training images
    for column_pointer, row in enumerate(keep_only_filtered_sorted_top_similarity_df.itertuples()):
        # train_image_path = f"path_to_train_images/{row.Train_Image}_image.nii.gz"  # Replace with actual path
        train_image = read_image(train_im_dir, row.Train_Image + "_image.nii.gz")
        if blend: 
                train_mask = read_image(train_im_dir, row.Train_Image + "_mask.nii.gz")
                train_image = color_blend(train_image, train_mask, mask_color=mask_color, transparency_factor=transparency_factor)
    
        ax = axes[row_pointer+1, column_pointer + 1]  # First column is for the test image
        ax.imshow(train_image, cmap='gray')
        # ax.set_title(f"Train {column_pointer + 1}, Label: {row.Train_Label}, "
        #                       f"CV: {row.Model_Index+1}, "
        #                       f"Cosine: {row.Cosine_Similarity:.4f}, MSE: {row.MSE:.4f}")
        ax.set_title(f"{row.Train_Image}\n Label: {row.Train_Label}, "
                          f"CV: {row.Model_Index+1}, "
                          f"Cosine: {row.Cosine_Similarity:.4f}, MSE: {row.MSE:.4f}")
        ax.axis('off')
    
    # Adjust layout and save the combined figure
    plt.tight_layout(rect=[0, 0, 1, 0.9])
    plt.savefig(os.path.join(save_dir, f"{patient_name}.png"))
    if visualize: plt.show()
    
    "Part IV: Count frequency of training patients appeared in image-level top similarities."
    frequency_df = sorted_top_similarity_df['Train_Patient_Name'].value_counts()
    frequency_df = frequency_df.reset_index()
    frequency_df.columns = ['Train_Patient_Name', 'Frequency']

    return top_similarity_df, filtered_sorted_top_similarity_df, frequency_df
        # top_similarity_df: store top similarities for each model at the image-level
        # filtered_sorted_top_similarity_df: top patient-level similarities from the image-level similarities. Training patient names are added
        # frequency_df: how many times a training patient appears in the top image-level similarities


In [23]:
def get_patient_info(patients_info, target_patient_names, keys, save_dir):
    """
    This function extracts patients' clinical info. 
    
    Args
    ==============
    patients_info: Patients' clinical info, csv file.
    target_patient_names: List, the names of the target patients. 
    keys: Keys (features) that will be extracted.
    save_dir: Directory to save the extracted clinical info.

    Returns
    ==============
    target_patient_info: A dataframe containing clinical info of the target patients
    """
    
    # Keep keys only, remove other columns
    filtered_patients_info = patients_info[keys]

    # # Patient names
    # target_patient_names = patient_level_similarity.iloc[0:3]["Train_Image"].tolist()
    
    target_patients_info = filtered_patients_info.loc[filtered_patients_info["Filename"].isin(target_patient_names)]
    
    # # Save the result
    # target_patient_info.to_excel(save_dir, index=False)

    return target_patients_info
    

In [24]:
def cal_pca(patient_name:str, 
            train_df, 
            test_df,
            top_similarity_df,
            save_dir:str,
            train_labelsize:list=[198, 159],
            visualize:bool=False):

    """
    This function calculates PCA. It collects feature embeddings from the training dataframe. It also collects feature
    embeddings of the given patient from the test dataframe. Once all the feature embeddings are collected, it calculates
    top 2 PCs. It plots training feature embeddings as benign and malignant with different colors. It also plots top training
    feature embeddings with different colors to separate them from the remaining training feature embeddings. Finally, it plots
    test feature embeddings. 

    Args:
    ====================
    patient_name: Name of the patient (e.g. "PN0085_8960_20190103")
    train_df: It is the dataframe that contains training image names, labels, model indices, and features.
    test_df: It is the dataframe that contains training image names, labels, predictions, model indices, and features.
    top_similarity_df: It is the dataframe that contains patient-level similarity scores for all models.
    save_dir: Directory to save the PCA plots.
    train_labelsize: It is a list that contains no. of benign and malignant images in the training dataset.
    visualize: Whether to visualize the plot.    
    """

    # Group training data by model_idx
    groupby_modelidx_train = train_df.groupby("Model_Index")
    n_groupby_modelidx = groupby_modelidx_train.ngroups  # no. of groups based on model_idx
    # print("No. of groups based on model_idx:", n_groupby_modelidx)
    
    # Create subplots arranged horizontally
    fig, axs = plt.subplots(1, n_groupby_modelidx, figsize=(6 * n_groupby_modelidx, 6), constrained_layout=True)
    fig.suptitle(f'PCA Visualization for Patient: {patient_name}', fontsize=16)
    
    # If there's only one subplot, wrap axs in a list for consistent indexing
    if n_groupby_modelidx == 1: axs = [axs]

    cnt_benign, cnt_malignant = [], []
    
    for model_idx in range(n_groupby_modelidx):
        # Prepare test features
        patient_data = test_df.loc[(test_df['Model_Index'] == model_idx) & (test_df['Name'] == patient_name)]
        test_features = patient_data.iloc[:, 4:] # first four columns are not features
    
        # Prepare training features. 
        group_modelidx_train = groupby_modelidx_train.get_group(model_idx).reset_index(drop=True) # collect information for the current model index
        train_features = group_modelidx_train.iloc[:, 3:]  # first three columns are not features
    
        # Combine train and test features
        combined_features = np.vstack([train_features, test_features])
        
        # Standardize the combined features
        scaler = StandardScaler()
        combined_features = scaler.fit_transform(combined_features)
    
        # Extract top training image names in a list from the similarity dataframe for the current model index
        top_trainnames = top_similarity_df[top_similarity_df["Model_Index"] == model_idx]["Train_Image"].tolist()
    
        # Find indices of the top training images in group_modelidx_train dataframe
        indices = group_modelidx_train.index[group_modelidx_train['Name'].isin(top_trainnames)].tolist()
        
        "Apply PCA to reduce dimensions"
        pca = PCA(n_components=2)
        pca_embeddings = pca.fit_transform(combined_features)
        
        # Separate transformed train and test embeddings
        train_pca_embeddings_benign = pca_embeddings[:train_labelsize[0]] # train benign
        train_pca_embeddings_malignant = pca_embeddings[train_labelsize[0]:len(train_features)] # train malignant
        test_pca_embeddings = pca_embeddings[len(train_features):] # test image
        
        # Extract PCA-transformed embeddings for the highlighted indices (top similarities)
        top_benign = train_pca_embeddings_benign[np.array(indices)[np.array(indices) < train_labelsize[0]]]
        top_malignant = train_pca_embeddings_malignant[np.array(indices)[np.array(indices) >= train_labelsize[0]] - train_labelsize[0]]

        cnt_benign.append(len(top_benign))
        cnt_malignant.append(len(top_malignant))
        
        print(f"Variance ratio for model {model_idx}:", pca.explained_variance_ratio_)
        
        "Scatter plot for PCA-transformed embeddings"
        # Plot training images as benign or malignant
        axs[model_idx].scatter(train_pca_embeddings_benign[:, 0], train_pca_embeddings_benign[:, 1], alpha=0.5, label="Train_benign", color='blue')
        axs[model_idx].scatter(train_pca_embeddings_malignant[:, 0], train_pca_embeddings_malignant[:, 1], alpha=0.5, label="Train_malignant", color='orange')
        
        # Highlight the specified training images
        axs[model_idx].scatter(top_benign[:, 0], top_benign[:, 1], label="Top_benign", color='cyan', edgecolors='black', linewidths=2, s=50)
        axs[model_idx].scatter(top_malignant[:, 0], top_malignant[:, 1], label="Top_malignant", color='gold', edgecolors='black', linewidths=2, s=50)
    
        # Plot the test image
        axs[model_idx].scatter(test_pca_embeddings[:, 0], test_pca_embeddings[:, 1], alpha=0.5, label="Test", marker='o', color='red', edgecolors='black', linewidths=2, s=100)
        
        axs[model_idx].legend()    
        axs[model_idx].set_title(f'PCA Visualization for Model Index {model_idx}')
        axs[model_idx].set_xlabel('Principal Component 1')
        axs[model_idx].set_ylabel('Principal Component 2')
    
    # Show the combined plot
    plt.savefig(os.path.join(save_dir, f"{patient_name}_pca.png")) 
    if visualize: plt.show()

    return cnt_benign, cnt_malignant

In [25]:
def cal_tsne(patient_name:str, 
            train_df, 
            test_df,
            top_similarity_df,
            save_dir:str,
            train_labelsize:list=[198, 159],
            perplexity:int=50, 
            n_iter:int=1000,
            visualize:bool=False):

    """
    This function calculates t-SNE. It collects feature embeddings from the training dataframe. It also collects feature
    embeddings of the given patient from the test dataframe. Once all the feature embeddings are collected, it calculates
    top 2 t-SNE components. It plots training feature embeddings as benign and malignant with different colors. It also plots top training
    feature embeddings with different colors to separate them from the remaining training feature embeddings. Finally, it plots
    test feature embeddings. 

    Args:
    ====================
    patient_name: Name of the patient (e.g. "PN0085_8960_20190103")
    train_df: It is the dataframe that contains training image names, labels, model indices, and features.
    test_df: It is the dataframe that contains training image names, labels, predictions, model indices, and features.
    top_similarity_df: It is the dataframe that contains patient-level similarity scores for all models.
    save_dir: Directory to save the PCA plots.
    train_labelsize: It is a list that contains no. of benign and malignant images in the training dataset.
    perplexity: It influences how many neighbors are considered when estimating the local data distribution.
                It typically considers about 3×perplexity nearest neighbors during the computation of conditional probabilities. 
                Small datasets (<1000 points): Lower perplexity (5–30). Large datasets: Higher perplexity (30–50).
    n_iter: No. of iterations to perform t-SNE.
    visualize: Whether to visualize the plot.    
    """    

    # Group training data by model_idx
    groupby_modelidx_train = train_df.groupby("Model_Index")
    n_groupby_modelidx = groupby_modelidx_train.ngroups  # no. of groups based on model_idx
    
    # Create subplots arranged horizontally
    fig, axs = plt.subplots(1, n_groupby_modelidx, figsize=(6 * n_groupby_modelidx, 6), constrained_layout=True)
    fig.suptitle(f't-SNE Visualization for Patient: {patient_name}', fontsize=16)
    
    # If there's only one subplot, wrap axs in a list for consistent indexing
    if n_groupby_modelidx == 1: axs = [axs]

    cnt_benign, cnt_malignant = [], [] # count #benign and #malignant
    
    for model_idx in range(n_groupby_modelidx):
        # Prepare test features
        patient_data = test_df.loc[(test_df['Model_Index'] == model_idx) & (test_df['Name'] == patient_name)]
        test_features = patient_data.iloc[:, 4:]  # first four columns are not features
    
        # Prepare training features
        group_modelidx_train = groupby_modelidx_train.get_group(model_idx).reset_index(drop=True)
        train_features = group_modelidx_train.iloc[:, 3:]  # first three columns are not features
    
        # Combine train and test features
        combined_features = np.vstack([train_features, test_features])
        
        # Standardize the combined features
        scaler = StandardScaler()
        combined_features = scaler.fit_transform(combined_features)
    
        # Extract top training image names in a list from the similarity dataframe for the current model index
        top_trainnames = top_similarity_df[top_similarity_df["Model_Index"] == model_idx]["Train_Image"].tolist()
    
        # Find indices of the top training images in group_modelidx_train dataframe
        indices = group_modelidx_train.index[group_modelidx_train['Name'].isin(top_trainnames)].tolist()
        
        "Apply t-SNE to reduce dimensions"
        tsne = TSNE(n_components=2, perplexity=perplexity, n_iter=n_iter, random_state=42)
        tsne_embeddings = tsne.fit_transform(combined_features)
        
        # Separate transformed train and test embeddings
        train_tsne_embeddings_benign = tsne_embeddings[:train_labelsize[0]]  # train benign
        train_tsne_embeddings_malignant = tsne_embeddings[train_labelsize[0]:len(train_features)]  # train malignant
        test_tsne_embeddings = tsne_embeddings[len(train_features):]  # test image
        
        # Extract t-SNE-transformed embeddings for the highlighted indices (top similarities)
        top_benign = train_tsne_embeddings_benign[np.array(indices)[np.array(indices) < train_labelsize[0]]]
        top_malignant = train_tsne_embeddings_malignant[np.array(indices)[np.array(indices) >= train_labelsize[0]] - train_labelsize[0]]

        cnt_benign.append(len(top_benign))
        cnt_malignant.append(len(top_malignant))
        
        "Scatter plot for t-SNE-transformed embeddings"
        # Plot training images as benign or malignant
        axs[model_idx].scatter(train_tsne_embeddings_benign[:, 0], train_tsne_embeddings_benign[:, 1], alpha=0.5, label="Train_benign", color='blue')
        axs[model_idx].scatter(train_tsne_embeddings_malignant[:, 0], train_tsne_embeddings_malignant[:, 1], alpha=0.5, label="Train_malignant", color='orange')
        
        # Highlight the specified training images
        axs[model_idx].scatter(top_benign[:, 0], top_benign[:, 1], label="Top_benign", color='cyan', edgecolors='black', linewidths=2, s=50)
        axs[model_idx].scatter(top_malignant[:, 0], top_malignant[:, 1], label="Top_malignant", color='gold', edgecolors='black', linewidths=2, s=50)
    
        # Plot the test image
        axs[model_idx].scatter(test_tsne_embeddings[:, 0], test_tsne_embeddings[:, 1], alpha=0.5, label="Test", marker='o', color='red', edgecolors='black', linewidths=2, s=100)
        
        axs[model_idx].legend()
        axs[model_idx].set_title(f't-SNE Visualization for Model Index {model_idx}')
        axs[model_idx].set_xlabel('t-SNE Dimension 1')
        axs[model_idx].set_ylabel('t-SNE Dimension 2')
    
    # Show the combined plot
    plt.savefig(os.path.join(save_dir, f"{patient_name}_tsne.png")) 
    if visualize: plt.show()

    return cnt_benign, cnt_malignant

### Create dataframe for training features

In [26]:
collect_data = []  # Use a list to collect all rows before creating the DataFrame
for data_per_model in store_train_features:
    for data in data_per_model: 
        # fromat of data: name, label, features, model_idx
        # Create a dictionary for the current row
        row_dict = {
            "Name": data[0],
            "Label": data[1],
            "Model_Index": data[3], 
        }
        
        # Add embeddings as key-value pairs to the dictionary
        row_dict.update({f"Feature_{j}": value for j, value in enumerate(data[2])})

        # Append the row dictionary to the list
        collect_data.append(row_dict)


# Convert the list of dictionaries to a DataFrame
train_df = pd.DataFrame(collect_data)


### Create dataframe for test features

In [27]:
collect_data = []  # Use a list to collect all rows before creating the DataFrame
for data_per_model in store_test_features:
    for data in data_per_model: 
        # fromat of data: name, label, features, model_idx
        # Create a dictionary for the current row
        row_dict = {
            "Name": data[0],
            "Label": data[1],
            "Prediction": data[3],
            "Model_Index": data[4], 
        }
        
        # Add embeddings as key-value pairs to the dictionary
        row_dict.update({f"Feature_{j}": value for j, value in enumerate(data[2])})

        # Append the row dictionary to the list
        collect_data.append(row_dict)


# Convert the list of dictionaries to a DataFrame
test_df = pd.DataFrame(collect_data)


#### Run

In [ ]:
keep_range = (0,4) # (354,356)

test_names = ["XYZ_1", "XYZ_2", "XYZ_3"] # Replace with actual test names

# Create dataframes to store confidence scores
df_pca, df_tsne, df_models = [], [], [] # the list will be converted to dataframe after all iterations

for i, patient_name in enumerate(test_names):
    print(f"{i}. Processing {patient_name}")

    # Patient label and prediction
    test_pred = int(test_results.loc[test_results["Names"]==patient_name]["Prediction"].values)
    test_label = int(test_results.loc[test_results["Names"]==patient_name]["Label"].values)

    # Not saving the dataframes. 
    top_similarity_df, filtered_sorted_top_similarity_df, frequency_df = post_similarity(detailed_results_df, patient_name, N_FOLDS,
                train_im_dir, test_im_dir, test_save_dir, 
                sort_by='Cosine_Similarity', keep_range=keep_range, 
                blend=False, visualize=False)

    "Uncomment to collect patient-label info"
    # # Read patients_info csv file
    # patients_info_file = "/research/m324371/Project/adnexal/radiomic_and_clinical_files/LabelStudio_DICOM_redCap_results.csv" 
    # patients_info = pd.read_csv(patients_info_file)

    # # Extract target patient names (test + top patient-level training images)
    # target_patient_names = [patient_name] +  filtered_sorted_top_similarity_df.iloc[0:3]["Train_Image"].tolist()

    # # Keys (features) to extract
    # keys = ["Filename", "dimension 2", "dimension 3", "Other solid component?", "Largest solid component max diameter", "lesion category", "internal wall",
    #     "Simple cyst? (unilocular, smooth internal wall, anechoic)", "acoustic shadows", "Color score", "Maximum dimension"]

    # # Save directory
    # save_dir = os.path.join(test_save_dir, patient_name + "_" + str(keep_range) + ".xlsx")

    # # Run the function
    # target_patients_info = get_patient_info(patients_info, target_patient_names, keys, save_dir)

    # # Save the result
    # target_patients_info.to_excel(save_dir, index=False)

    "Uncomment to calculate PCA"
    cnt_benign, cnt_malignant = cal_pca(patient_name=patient_name, 
                                        train_df=train_df, 
                                        test_df=test_df,
                                        top_similarity_df=top_similarity_df,
                                        save_dir=test_save_dir,
                                        train_labelsize=[198, 159],
                                        visualize=False)
    
    # PCA confidence score
    pca_confidence = sum(cnt_benign)/(sum(cnt_benign)+sum(cnt_malignant)) if test_label==0 else sum(cnt_malignant)/(sum(cnt_benign)+sum(cnt_malignant))
    pca_row = {"Name":patient_name, "Label":test_label, "Prediction":test_pred, "Cnt_benign":cnt_benign, "Cnt_malignant":cnt_malignant, "Confidence":pca_confidence}    
    df_pca.append(pca_row)

    "Uncomment to calculate t-SNE"
    cnt_benign, cnt_malignant = cal_tsne(patient_name=patient_name, 
                                        train_df=train_df, 
                                        test_df=test_df,
                                        top_similarity_df=top_similarity_df,
                                        save_dir=test_save_dir,
                                        train_labelsize=[198, 159],
                                        perplexity=50,
                                        n_iter=1000,
                                        visualize=False)

    # t-SNE confidence score
    tsne_confidence = sum(cnt_benign)/(sum(cnt_benign)+sum(cnt_malignant)) if test_label==0 else sum(cnt_malignant)/(sum(cnt_benign)+sum(cnt_malignant))
    tsne_row = {"Name":patient_name, "Label":test_label, "Prediction":test_pred, "Cnt_benign":cnt_benign, "Cnt_malignant":cnt_malignant, "Confidence":tsne_confidence}    
    df_tsne.append(tsne_row)

    "Uncomment to calculated confidence score w.r.t. models"
    model_outputs = test_df[test_df["Name"]==patient_name]["Prediction"].tolist() # output is like [0,0,1,0,1]
    n_benign, n_malignant = model_outputs.count(0), model_outputs.count(1)
    model_confidence = n_benign/(n_benign+n_malignant) if test_label==0 else n_malignant/(n_benign+n_malignant)
    model_row = {"Name":patient_name, "Label":test_label, "Prediction":test_pred, "Outputs":model_outputs, "Confidence":model_confidence}    
    df_models.append(model_row)
    
# Convert list to dataframe
df_pca = pd.DataFrame(df_pca)
df_tsne = pd.DataFrame(df_tsne)
df_models = pd.DataFrame(df_models)

# Save the dataframes
df_pca.to_excel(os.path.join(test_save_dir, "pca_score.xlsx"), index=False)
df_tsne.to_excel(os.path.join(test_save_dir, "tsne_score.xlsx"), index=False)
df_models.to_excel(os.path.join(test_save_dir, "models_score.xlsx"), index=False)

print("Processing done")

### MAP@K for image-to-image retrieval 

In [ ]:
def precision_at_k(relevant_labels, k):
    """Compute precision at K"""
    relevant_k = relevant_labels[:k]  # Top K retrieved results
    return np.sum(relevant_k) / k  # Fraction of relevant instances

def average_precision_at_k(relevant_labels, k):
    """Compute AP@K for a single test image"""
    num_relevant = np.sum(relevant_labels[:k])  # Count relevant results in top K
    if num_relevant == 0:
        return 0
    ap_k = sum(precision_at_k(relevant_labels, i+1) for i in range(k) if relevant_labels[i]) / num_relevant
    return ap_k

def mean_average_precision_at_k(df, k):
    """Compute MAP@K for each Model_Index"""
    map_scores = {}

    for model_idx, model_df in df.groupby("Model_Index"):
        ap_scores = []
        
        for test_image, group in model_df.groupby("Test_Image"):
            sorted_group = group.sort_values(by="Cosine_Similarity", ascending=False)  # Sort by similarity
            relevant_labels = (sorted_group["Test_Label"] == sorted_group["Train_Label"]).astype(int).values  # Binary relevance
            ap_k = average_precision_at_k(relevant_labels, k)
            ap_scores.append(ap_k)
        
        map_scores[model_idx] = np.mean(ap_scores)  # Compute MAP@K for the model
    
    return map_scores

# Uncomment to load detailed_results_df from excel file
# file_path = "similarity_test_v7-3_top5/train_test_similarity_2025-01-28_20-45-36.xlsx"
# detailed_results_df = pd.read_excel(file_path)

K = 50  # Define the value of K

# Compute MAP@K for each model
map_at_k_results = mean_average_precision_at_k(detailed_results_df, K)

# Print results
for model_idx, map_k in map_at_k_results.items():
    print(f"Model {model_idx} - MAP@{K}: {map_k:.4f}")
